<a href="https://colab.research.google.com/github/MiroPol21/dspracticum2025/blob/main/get_articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Fetch 500-1000 abstracts**

In [ ]:
# pip install feedparser

In [8]:
import feedparser, json, time, re, requests

In [ ]:
queries = ["recreation", "leisure", "tourism", "sports", "wellbeing", "outdoor", "play"]
max_per_query = 150  # keep it small
results = []

for query in queries:
    print(f"Fetching {query}")
    url = f"http://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results={max_per_query}"
    feed = feedparser.parse(url)
    for entry in feed.entries:
        text = (entry.get("title", "") + " " + entry.get("summary", "")).strip()
        if len(text) > 300:  # skip junk
            results.append({"text": text})
    time.sleep(2)

with open("arxiv_recreology_small.jsonl", "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r) + "\n")

print(f"Saved {len(results)} abstracts (~{sum(len(r['text']) for r in results):,} chars)")

**2. Clean**

In [4]:
cleaned = []
with open("arxiv_recreology_small.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        d = json.loads(line)
        text = re.sub(r"\s+", " ", d["text"])
        text = re.sub(r"\(arXiv:[^)]+\)", "", text)
        cleaned.append({"text": text.strip()})

with open("recreology_clean.jsonl", "w", encoding="utf-8") as f:
    for c in cleaned:
        f.write(json.dumps(c) + "\n")

print(f"Cleaned {len(cleaned)} entries.")

Cleaned 1036 entries.


**3. Verify**

In [ ]:
with open("arxiv_recreology_small.jsonl", "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        data = json.loads(line)
        print(f"--- ARTICLE {i+1} ---")
        print(data["text"][:1000])  # first 1000 chars
        print()
        if i >= 4:  # show 5 examples
            break

In [6]:
char_count = 0
line_count = 0

with open("arxiv_recreology_small.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        text = data.get("text", "")
        char_count += len(text)
        line_count += 1

print(f"Total characters: {char_count:,}")
print(f"Number of articles: {line_count}")
print(f"Average per article: {char_count // max(line_count, 1):,}")


Total characters: 1,235,733
Number of articles: 1036
Average per article: 1,192


**4. Expand**

In [7]:
import requests, json

topics = ["Recreation", "Leisure studies", "Tourism", "Outdoor education", "Play (activity)", "Sports sociology"]
data = []

for topic in topics:
    r = requests.get(f"https://en.wikipedia.org/api/rest_v1/page/plain/{topic}")
    if r.status_code == 200:
        data.append({"text": r.text})

with open("wiki_recreology.jsonl", "w", encoding="utf-8") as f:
    for d in data:
        f.write(json.dumps(d) + "\n")

print(f"Added {len(data)} Wikipedia articles.")

Added 0 Wikipedia articles.
